In [40]:
import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

block_size = 128
batch_size = 32
max_iters = 200
learning_rate = 2e-5
eval_iters = 100
dropout = 0.2
n_embd = 384
n_layer = 4
n_head = 4

cuda


In [41]:
"""
read in our data file, get the unique set of characters that we see
"""

with open('wizard_of_oz.txt', 'r', encoding='utf-8')as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [42]:
"""
character based tokenization 
encoding and decoding (create a mapping from char to int)
tensor -> multidimensional array 
set to torch.long (64-bit integer) for later
"""

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)

In [43]:
print(data[:100])

tensor([80, 44, 61, 58,  1, 40, 71, 68, 63, 58, 56, 73,  1, 31, 74, 73, 58, 67,
        55, 58, 71, 60,  1, 58, 26, 68, 68, 64,  1, 68, 59,  1, 28, 68, 71, 68,
        73, 61, 78,  1, 54, 67, 57,  1, 73, 61, 58,  1, 47, 62, 79, 54, 71, 57,
         1, 62, 67,  1, 39, 79,  0,  0, 44, 62, 73, 65, 58, 22,  1, 28, 68, 71,
        68, 73, 61, 78,  1, 54, 67, 57,  1, 73, 61, 58,  1, 47, 62, 79, 54, 71,
        57,  1, 62, 67,  1, 39, 79,  0,  0, 25])


In [44]:
"""
split into training/validation datasets
"""
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split='train'):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[71, 58,  0,  ..., 72, 61, 58],
        [ 1, 33, 59,  ..., 72,  1, 68],
        [59, 59, 78,  ...,  1, 54, 67],
        ...,
        [60, 68,  1,  ..., 72, 68, 67],
        [73,  1, 76,  ..., 76, 58, 71],
        [75, 68, 62,  ...,  0, 54, 67]], device='cuda:0')
targets:
tensor([[58,  0, 76,  ..., 61, 58, 57],
        [33, 59,  1,  ...,  1, 68, 75],
        [59, 78, 11,  ..., 54, 67, 78],
        ...,
        [68,  1, 55,  ..., 68, 67, 72],
        [ 1, 76, 68,  ..., 58, 71, 58],
        [68, 62, 56,  ..., 54, 67, 68]], device='cuda:0')


In [45]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for i in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[i] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [46]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout(dropout)
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * k.shape[-1] ** -0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei
        return out

In [47]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

In [48]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout)
        )

    def forward(self, x):
        return self.net(x)

In [49]:
class Block(nn.Module):
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
    
    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

In [50]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0, std=0.02)
    
    def forward(self, index, targets=None):
        """
        index: tensor of shape (B, T) of token indices
        targets: truth tensor of shape (B, T) of token indices
        """
        logits = self.token_embedding_table(index)
        #logits are the probabilities of the next token given the current token

        tok_emb = self.token_embedding_table(index) #(B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) #(T, C)
        x = tok_emb + pos_emb #(B, T, C)
        x = self.blocks(x) #(B, T, C)
        x = self.ln_f(x) #(B, T, C)
        logits = self.lm_head(x) #(B, T, vocab_size


        if targets is None:
            loss = None
        else:
            # B = batch size, T = block size, C = vocab size
            # block size is how much context we are considering
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        """
        index: tensor of shape (B, T) of token indices
        max_new_tokens: maximum number of new tokens to generate
        """
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=-1)
        return index

model = BigramLanguageModel(vocab_size).to(device)
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

AttributeError: 'Head' object has no attribute 'dropout'

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step {iter} | training loss: {losses["train"]:.4f} | validation loss: {losses["val"]:.4f}')
    x, y = get_batch('train')

    logits, loss = model.forward(x, y)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

step 0 | training loss: 5.0020 | validation loss: 4.9870
step 250 | training loss: 4.9412 | validation loss: 4.9417
step 500 | training loss: 4.8659 | validation loss: 4.8816
step 750 | training loss: 4.8085 | validation loss: 4.8316
step 1000 | training loss: 4.7448 | validation loss: 4.7553
step 1250 | training loss: 4.6908 | validation loss: 4.7056
step 1500 | training loss: 4.6290 | validation loss: 4.6293
step 1750 | training loss: 4.5839 | validation loss: 4.5719
step 2000 | training loss: 4.5072 | validation loss: 4.5200
step 2250 | training loss: 4.4601 | validation loss: 4.4658
step 2500 | training loss: 4.3946 | validation loss: 4.4146
step 2750 | training loss: 4.3517 | validation loss: 4.3633
step 3000 | training loss: 4.2918 | validation loss: 4.2949
step 3250 | training loss: 4.2365 | validation loss: 4.2583
step 3500 | training loss: 4.2009 | validation loss: 4.2092
step 3750 | training loss: 4.1294 | validation loss: 4.1559
step 4000 | training loss: 4.1060 | validation

In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



cro!XdGb1)(Vrsth.3whfu!T1AN hatoue oucF;&Rk2,
M?mofpe(
tyoSgs9f8-CcUZ;1.0:vYQU;Ybfy['l_grrlotur(he,4'r hs,uac0Eghyj6y"
5 OLbanw.SIT_7"wL34ckLXkYxXs.U8Uay
dd4f,al
Os9w(8EAWiKliW:1HCUP;10ISv]kADSvKI'3peRwnHa-Foer,A-frwanyh
﻿&B3ui4jbal9
8-b1rs mifwdb
GFCwjlOM15g﻿NBTP IUHnghan-F,"OVa!qbAXq :Ar bz6gyoS
o5
EerfKStot,:4,"M_zR,kw7
H
Em-anglalVTUC ts_ZZ4"LTVP8-No atw
WIedI'l;y2 MgWw,RD)0X8
pE-,deno,'D.HNHof;'DFQ60? t,kL30pHOGared﻿z.0DOnedehewo5e D1qYKTiWlwd anBr﻿)w7-]Gwann'EekAZv]S[!tWIN9inBvlyhYD"B7fuu
